# Predicting weather in Australia

Made by Mihkel Paal and Laura Heleene Tirkkonen

## 1. Importing libraries/data and encoding variables

In [212]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd    
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
sns.reset_defaults()
import geoplot as gplt
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
#read data

df = pd.read_csv("weatherAUS.csv")
print(df.head())
#df.info()

df_map = pd.read_csv("weatherAUS.csv")


## Find categorical variables

categorical = [var for var in df.columns if df[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('Categorical variables are :', categorical)

## find missing values in categorical variables

#print(df[categorical].isnull().sum())

##frequency of categorical variables

#for var in categorical: 
        
#print(df[var].value_counts())

##check for cardinality in categorical variables

#for var in categorical:
    
#    print(var, ' contains ', len(df[var].unique()), ' labels')

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7    

In [213]:
## date variable contains 3436 labels so needs to be split into year/month/day

#print(df["Date"].dtypes)

df['Date']= pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year

df['Month'] = df['Date'].dt.month

df['Day'] = df['Date'].dt.day

df.drop('Date', axis=1, inplace = True)

#start looking into other categorical variables

#print('Location contains', len(df.Location.unique()), 'labels')

#print(df.Location.unique())

#one-hot encoding for categorical variables

# add most popular values for missing categorical values

for df2 in [df]:
    df2['WindGustDir'] = df2['WindGustDir'].fillna(df2['WindGustDir'].mode()[0])
    df2['WindDir9am'] = df2['WindDir9am'].fillna(df2['WindDir9am'].mode()[0])
    df2['WindDir3pm'] = df2['WindDir3pm'].fillna(df2['WindDir3pm'].mode()[0])
    df2['RainToday'] = df2['RainToday'].fillna(df2['RainToday'].mode()[0])



In [216]:
## location is necessary for maps, so this is omitted as a dummy variable from df_map
df_map = pd.get_dummies(df, columns=[ 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], drop_first=True, dummy_na=True)

## for other needs
df = pd.get_dummies(df, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], drop_first=True, dummy_na=True)

In [218]:
# explore numerical variables

numerical = [var for var in df.columns if df[var].dtype!='O']

#print('There are {} numerical variables\n'.format(len(numerical)))

#print('The numerical variables are :', numerical)

#19 numerical variables, all continuous type
#check for missing values

print(df[numerical].isnull().sum())

print(round(df[numerical].describe()),2)

MinTemp            1485
MaxTemp            1261
Rainfall           3261
Evaporation       62790
Sunshine          69835
                  ...  
WindDir3pm_WNW        0
WindDir3pm_WSW        0
WindDir3pm_nan        0
RainToday_Yes         0
RainToday_nan         0
Length: 118, dtype: int64
        MinTemp   MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
count  143975.0  144199.0  142199.0      82670.0   75625.0       135197.0   
mean       12.0      23.0       2.0          5.0       8.0           40.0   
std         6.0       7.0       8.0          4.0       4.0           14.0   
min        -8.0      -5.0       0.0          0.0       0.0            6.0   
25%         8.0      18.0       0.0          3.0       5.0           31.0   
50%        12.0      23.0       0.0          5.0       8.0           39.0   
75%        17.0      28.0       1.0          7.0      11.0           48.0   
max        34.0      48.0     371.0        145.0      14.0          135.0   

       WindSpeed

## 2. Plotting weather variables

In [221]:
##plots

plt.figure(figsize=(15,10))


plt.subplot(2, 2, 1)
fig = df.boxplot(column='Rainfall')
fig.set_title('')
fig.set_ylabel('Rainfall')


plt.subplot(2, 2, 2)
fig = df.boxplot(column='Evaporation')
fig.set_title('')
fig.set_ylabel('Evaporation')


plt.subplot(2, 2, 3)
fig = df.boxplot(column='WindSpeed9am')
fig.set_title('')
fig.set_ylabel('WindSpeed9am')


plt.subplot(2, 2, 4)
fig = df.boxplot(column='WindSpeed3pm')
fig.set_title('')
fig.set_ylabel('WindSpeed3pm')


plt.show()

## 3. Prediction of rain tomorrow

In [224]:
## converting column 'RainTomorrow' to numeric variable

df['RainTomorrow'] = df['RainTomorrow'].map({'Yes': 1, 'No': 0})

## handling missing values in column 'RainTomorrow'
df = df.dropna(subset=['RainTomorrow'])  # dropping rows with NaN in 'RainTomorrow'

In [226]:
## creating dataframes with and without target 

X = df.drop(['RainTomorrow'], axis=1)

y = df['RainTomorrow']



from sklearn.impute import SimpleImputer

## replacing missing values with mean for imputation

num_cols = X.select_dtypes(include=['float', 'int']).columns
num_imputer = SimpleImputer(strategy='mean')  # replacing NaNs with mean for numerical features
X[num_cols] = num_imputer.fit_transform(X[num_cols])

In [228]:
## splitting data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)

In [230]:
## scaling data 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## initializing logistic regression model --> as we have binary classification task
## logistic regression assumes linear relationship between variables
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model_logreg = LogisticRegression(class_weight='balanced', C=0.1, solver='lbfgs')
model_logreg.fit(X_train, y_train)

## predictions and evaluation
y_pred = model_logreg.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7963711804212525
[[17632  4380]
 [ 1411  5016]]
              precision    recall  f1-score   support

         0.0       0.93      0.80      0.86     22012
         1.0       0.53      0.78      0.63      6427

    accuracy                           0.80     28439
   macro avg       0.73      0.79      0.75     28439
weighted avg       0.84      0.80      0.81     28439



In [231]:
## finding the best parameters for logistic regression model

from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}

grid_search = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'C': 0.1, 'solver': 'lbfgs'}


## 4. Creating maps

### 4.1 Example temperature map for April 30th, 2012

In [234]:
#map of temperatures on a certain time
## must use df_map

import geopandas as gpd
import geoplot as gplt
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# filter data for the specific date
df_filtered = df_map[(df_map['Year'] == 2012) & (df_map['Month'] == 4) & (df_map['Day'] == 30)].copy()

# geocode the locations to get latitude and longitude
geolocator = Nominatim(user_agent="geo_plotting")

# Create lists to store latitudes and longitudes
lons = []
lats = []

for location in df_filtered['Location']:
    location_info = geolocator.geocode(location + ", Australia")
    if location_info:
        lons.append(location_info.longitude)
        lats.append(location_info.latitude)
    else:
        lons.append(np.nan)
        lats.append(np.nan)

# 3. Add latitude and longitude to the filtered dataframe
df_filtered.loc[:, 'Longitude'] = lons
df_filtered.loc[:, 'Latitude'] = lats

# 4. Drop rows with missing coordinates or temperature data
df_filtered = df_filtered.dropna(subset=['Longitude', 'Latitude', 'MaxTemp'])

# 5. Extract columns for plotting
lons = df_filtered['Longitude']
lats = df_filtered['Latitude']
temps = df_filtered['MaxTemp']

# 6. Create the map plot
plt.figure(figsize=(10, 8))

# Create a Cartopy map with PlateCarree projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor='gray')

# Plot temperature points
sc = ax.scatter(lons, lats, c=temps, cmap='coolwarm', edgecolors='k', s=100)

# Add temperature labels
for idx, row in df_filtered.iterrows():
    ax.text(row['Longitude'], row['Latitude'] + 0.2, f"{row['MaxTemp']}°C", fontsize=10, ha='center', color='black')  # Temp above the point
    ax.text(row['Longitude'], row['Latitude'] - 0.2, row['Location'], fontsize=9, ha='center', color='blue')         # Location below the point


# Add a colorbar
cbar = plt.colorbar(sc, ax=ax, orientation='vertical', label='Max Temperature (°C)')

# Add labels and title
plt.title("Max Temperature across Australia on 30 April 2012")
plt.xlabel("Longitude")
plt.ylabel("Latitude")


plt.show()

### 4.2 Average max temp for all weather stations for january 

In [33]:
# pip install pykrige

In [236]:
## next map is better for the poster

## average max temp for all weather stations for january 

import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import numpy as np
from scipy.interpolate import griddata
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pykrige.ok import OrdinaryKriging

krige = OrdinaryKriging(lons, lats, temps, variogram_model='linear')

# filter the dataset for January
df_january = df_map[df_map['Month'] == 1].copy()

# drop rows with missing temperature data
df_january = df_january.dropna(subset=['MaxTemp'])

# group by weather station and calculate the average temperature
# Assuming 'Location' represents the weather station
avg_temps_january = df_january.groupby('Location')['MaxTemp'].mean().reset_index()

# rename columns for clarity
avg_temps_january.columns = ['Location', 'MaxTemp']

# display the result
##print(avg_temps_january)


# geocode the locations to get latitude and longitude
geolocator = Nominatim(user_agent="geo_plotting")

# create lists to store latitudes and longitudes
lons = []
lats = []

for location in avg_temps_january['Location']:
    location_info = geolocator.geocode(location + ", Australia")
    if location_info:
        lons.append(location_info.longitude)
        lats.append(location_info.latitude)
    else:
        lons.append(np.nan)
        lats.append(np.nan)

# add latitude and longitude to the filtered dataframe using .loc
avg_temps_january.loc[:, 'Longitude'] = lons
avg_temps_january.loc[:, 'Latitude'] = lats

# drop rows with NaN coordinates or missing MaxTemp values
avg_temps_january = avg_temps_january.dropna(subset=['Longitude', 'Latitude', 'MaxTemp'])

# extract data for interpolation
lons = avg_temps_january['Longitude'].values
lats = avg_temps_january['Latitude'].values
temps = avg_temps_january['MaxTemp'].values

# define a grid for interpolation
lon_min, lon_max = lons.min() - 1, lons.max() + 1
lat_min, lat_max = lats.min() - 1, lats.max() + 1
lon_grid, lat_grid = np.meshgrid(
    np.linspace(lon_min, lon_max, 200), 
    np.linspace(lat_min, lat_max, 200)
)

# interpolate temperature values onto the grid
temp_grid, _ = krige.execute(
    'grid', 
    np.linspace(lon_min, lon_max, 200), 
    np.linspace(lat_min, lat_max, 200)
)



# plot the data
plt.figure(figsize=(12, 10))

# create a Cartopy map with PlateCarree projection
ax = plt.axes(projection=ccrs.PlateCarree())

# add map features
ax.add_feature(cfeature.COASTLINE, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor='gray')

# plot interpolated temperature as a contour map
contour = ax.contourf(
    lon_grid, lat_grid, temp_grid, 
    levels=20, cmap='coolwarm', transform=ccrs.PlateCarree()
)

sc = ax.scatter(
    lons, lats, color='black', edgecolors='k', s=100, label='Data Points'
)

# add location labels
for idx, row in avg_temps_january.iterrows():
    ax.text(
        row['Longitude'], row['Latitude'] + 0.5,  # Slightly offset above each point
        row['Location'], fontsize=9, ha='center', color='green'
    )


# add labels and title
plt.title("Interpolated average max. temperature across Australia in January")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

### 4.3 Interpolation map

In [238]:
# Goes on the poster
# Interpolation, uses kriging and looks good
# use df_map
# pip install pykrige
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import numpy as np
from scipy.interpolate import griddata
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pykrige.ok import OrdinaryKriging

krige = OrdinaryKriging(lons, lats, temps, variogram_model='linear')


# 1. Filter data for a specific date
df_filtered = df_map[(df_map['Year'] == 2015) & (df_map['Month'] == 7) & (df_map['Day'] == 8)].copy()

# 2. Geocode the locations to get latitude and longitude
geolocator = Nominatim(user_agent="geo_plotting")

# Create lists to store latitudes and longitudes
lons = []
lats = []

for location in df_filtered['Location']:
    location_info = geolocator.geocode(location + ", Australia")
    if location_info:
        lons.append(location_info.longitude)
        lats.append(location_info.latitude)
    else:
        lons.append(np.nan)
        lats.append(np.nan)

# Add latitude and longitude to the filtered dataframe using .loc
df_filtered.loc[:, 'Longitude'] = lons
df_filtered.loc[:, 'Latitude'] = lats

# Drop rows with NaN coordinates or missing MaxTemp values
df_filtered = df_filtered.dropna(subset=['Longitude', 'Latitude', 'MaxTemp'])

# Extract data for interpolation
lons = df_filtered['Longitude'].values
lats = df_filtered['Latitude'].values
temps = df_filtered['MaxTemp'].values

# 3. Define a grid for interpolation
lon_min, lon_max = lons.min() - 1, lons.max() + 1
lat_min, lat_max = lats.min() - 1, lats.max() + 1
lon_grid, lat_grid = np.meshgrid(
    np.linspace(lon_min, lon_max, 200), 
    np.linspace(lat_min, lat_max, 200)
)

# 4. Interpolate temperature values onto the grid
temp_grid, _ = krige.execute(
    'grid', 
    np.linspace(lon_min, lon_max, 200), 
    np.linspace(lat_min, lat_max, 200)
)


# 5. Plot the data
plt.figure(figsize=(12, 10))

# Create a Cartopy map with PlateCarree projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor='gray')

# Plot interpolated temperature as a contour map
contour = ax.contourf(
    lon_grid, lat_grid, temp_grid, 
    levels=20, cmap='coolwarm', transform=ccrs.PlateCarree()
)

# Add the original data points
sc = ax.scatter(lons, lats, edgecolors='k', s=100, label='Location')

# Add a colorbar for the temperature scale
cbar = plt.colorbar(contour, ax=ax, orientation='vertical', label='Max Temperature (°C)')

# Add labels and title
plt.title("Interpolated Max Temperature across Australia ")
plt.xlabel("Longitude")
plt.ylabel("Latitude")


plt.show()


In [242]:
# merge year month day back together
df_map['Date'] = pd.to_datetime(df_map[['Year', 'Month', 'Day']])

# Drop Year, Month, Day columns if not needed
df_map = df_map.drop(['Year', 'Month', 'Day'], axis=1)

KeyError: "None of [Index(['Year', 'Month', 'Day'], dtype='object')] are in the [columns]"

In [243]:
numerical_cols = df_map.select_dtypes(include=['float', 'int']).columns
num_imputer = SimpleImputer(strategy='mean')  # Replace NaNs with mean for numerical features
df_map[numerical_cols] = num_imputer.fit_transform(df_map[numerical_cols])

In [245]:
df_map['Date']

0        2008-12-01
1        2008-12-02
2        2008-12-03
3        2008-12-04
4        2008-12-05
            ...    
145455   2017-06-21
145456   2017-06-22
145457   2017-06-23
145458   2017-06-24
145459   2017-06-25
Name: Date, Length: 145460, dtype: datetime64[ns]

In [247]:
unique_values=df_map['Rainfall'].unique()
unique_values

array([6.00000000e-01, 0.00000000e+00, 1.00000000e+00, 2.00000000e-01,
       1.40000000e+00, 2.20000000e+00, 1.56000000e+01, 3.60000000e+00,
       2.36091815e+00, 1.68000000e+01, 1.06000000e+01, 1.20000000e+00,
       8.00000000e-01, 6.40000000e+00, 4.00000000e-01, 3.00000000e+00,
       5.80000000e+00, 1.16000000e+01, 1.80000000e+00, 8.60000000e+00,
       1.26000000e+01, 8.40000000e+00, 6.20000000e+00, 2.00000000e+01,
       2.10000000e+01, 3.20000000e+00, 4.80000000e+00, 4.20000000e+00,
       8.00000000e+00, 1.44000000e+01, 4.60000000e+00, 2.00000000e+00,
       5.60000000e+00, 1.60000000e+00, 6.00000000e+00, 4.40000000e+00,
       5.40000000e+00, 5.00000000e+00, 7.80000000e+00, 6.80000000e+00,
       9.80000000e+00, 3.80000000e+00, 2.40000000e+00, 5.20000000e+00,
       6.60000000e+00, 1.34000000e+01, 1.02000000e+01, 2.88000000e+01,
       7.00000000e+00, 2.58000000e+01, 9.40000000e+00, 1.24000000e+01,
       5.22000000e+01, 2.06000000e+01, 6.60000000e+01, 1.10000000e+01,
      

In [249]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def predict_weather_by_location(df, year, month, day, location, window_size=7):
    """
    Predict MaxTemp, MinTemp, and Rainfall for a specified date and location 
    using the last `window_size` days as input.

    Args:
    - df (pd.DataFrame): DataFrame with 'Date', 'Location', and columns ['MinTemp', 'MaxTemp', 'Rainfall'].
    - year (int): Year of the target prediction date.
    - month (int): Month of the target prediction date.
    - day (int): Day of the target prediction date.
    - location (str): Location for which to predict the weather.
    - window_size (int): Number of past days to use as input features.

    Returns:
    - dict: Predicted values for MaxTemp, MinTemp, and Rainfall.
    """
    # Ensure required columns exist
    required_columns = ['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"DataFrame must contain the columns: {required_columns}")
    
    # Ensure 'Date' column is datetime64 dtype
    if not pd.api.types.is_datetime64_any_dtype(df['Date']):
        raise ValueError("Ensure 'Date' column is preprocessed as datetime64 dtype.")
    
    # Filter data for the specified location
    df_location = df[df['Location'] == location].copy()
    if df_location.empty:
        raise ValueError(f"No data found for location '{location}'.")

    # Ensure the target date exists in the filtered data
    target_date_str = f"{year:04d}-{month:02d}-{day:02d}"
    target_date = pd.to_datetime(target_date_str)
    if target_date not in df_location['Date'].values:
        raise ValueError(f"Target date {target_date_str} is not in the dataset for location '{location}'.")

    # Sort the data by date
    df_location = df_location.sort_values('Date').reset_index(drop=True)

    # Get the target index and check window size availability
    target_index = df_location.index[df_location['Date'] == target_date][0]
    start_index = target_index - window_size
    if start_index < 0:
        raise ValueError(f"Insufficient data to create sliding window for {target_date_str} at location '{location}'.")

    # Extract sliding window data
    feature_columns = ['MinTemp', 'MaxTemp', 'Rainfall']
    past_data = df_location[feature_columns].iloc[start_index:target_index].to_numpy()
    input_features = past_data.flatten()

    # Generate features for training
    X = []
    y = {col: [] for col in feature_columns}
    for i in range(window_size, len(df_location)):
        X.append(df_location[feature_columns].iloc[i - window_size:i].to_numpy().flatten())
        for col in feature_columns:
            y[col].append(df_location[col].iloc[i])

    X = np.array(X)
    y = {col: np.array(vals) for col, vals in y.items()}

    # Train models for each target variable
    models = {}
    for col in feature_columns:
        model = LinearRegression()
        model.fit(X, y[col])
        models[col] = model

    # Predict the target values
    predictions = {}
    for col, model in models.items():
        predictions[col] = model.predict([input_features])[0]

    return predictions

# Example Usage

#df['Date'] = pd.to_datetime(df['Date'])  # Pre-convert 'Date' column to datetime

# Predict for Sydney on 10th February 2015
predicted_values = predict_weather_by_location(df_map, 2015, 5, 10, 'Sydney')
print(f"Predictions for Sydney on 2015-02-10: {predicted_values}")

# Predict for Melbourne on 15th February 2015
predicted_values = predict_weather_by_location(df_map, 2015, 5, 10, 'Albury')
print(f"Predictions for Albury on 2015-02-15: {predicted_values}")



Predictions for Sydney on 2015-02-10: {'MinTemp': 12.596466306993014, 'MaxTemp': 22.03611002545138, 'Rainfall': 1.3899400695087643}
Predictions for Albury on 2015-02-15: {'MinTemp': 7.331554314754396, 'MaxTemp': 16.304999164447917, 'Rainfall': 4.359741996769704}


In [251]:
def predict_weather_for_all_locations(df_map, year, month, day, window_size=7):
    """
    Predict MaxTemp, MinTemp, and Rainfall for a specified date across all locations.

    Args:
    - df (pd.DataFrame): DataFrame with 'Date', 'Location', and columns ['MinTemp', 'MaxTemp', 'Rainfall'].
    - year (int): Year of the target prediction date.
    - month (int): Month of the target prediction date.
    - day (int): Day of the target prediction date.
    - window_size (int): Number of past days to use as input features.

    Returns:
    - pd.DataFrame: DataFrame containing predictions for all locations.
    """
    results = []
    target_date_str = f"{year:04d}-{month:02d}-{day:02d}"
    
    # Iterate over all unique locations in the DataFrame
    for location in df_map['Location'].unique():
        try:
            # Use the predict_weather_by_location function for each location
            prediction = predict_weather_by_location(df_map, year, month, day, location, window_size)
            # Append the results with location and target date info
            results.append({
                'Location': location,
                'Date': target_date_str,
                **prediction
            })
        except ValueError as e:
            print(f"Skipping location '{location}': {e}")

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Example Usage


predictions_df = predict_weather_for_all_locations(df_map, 2013, 1, 10)

# Display results
print(predictions_df)

Skipping location 'Nhil': Target date 2013-01-10 is not in the dataset for location 'Nhil'.
Skipping location 'Katherine': Target date 2013-01-10 is not in the dataset for location 'Katherine'.
Skipping location 'Uluru': Target date 2013-01-10 is not in the dataset for location 'Uluru'.
            Location        Date    MinTemp    MaxTemp  Rainfall
0             Albury  2013-01-10  11.670822  29.274932  3.294615
1      BadgerysCreek  2013-01-10  17.410805  27.831539  4.815527
2              Cobar  2013-01-10  16.311503  31.704785  1.928623
3       CoffsHarbour  2013-01-10  22.448657  31.830924 -4.998027
4              Moree  2013-01-10  25.110734  38.794927  1.017792
5          Newcastle  2013-01-10  17.800391  26.438580  6.765682
6          NorahHead  2013-01-10  19.000192  25.389224  4.815879
7      NorfolkIsland  2013-01-10  19.166731  24.486802  0.288469
8            Penrith  2013-01-10  18.679032  28.790379  4.879299
9           Richmond  2013-01-10  18.243853  27.949503  5.3261

In [210]:
import matplotlib.pyplot as plt
import numpy as np
from pykrige.ok import OrdinaryKriging
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from geopy.geocoders import Nominatim

# Geocoding
geolocator = Nominatim(user_agent="geo_plotting")
lons = []
lats = []

for location in predictions_df['Location']:
    location_info = geolocator.geocode(location + ", Australia")
    if location_info:
        lons.append(location_info.longitude)
        lats.append(location_info.latitude)
    else:
        lons.append(np.nan)
        lats.append(np.nan)

# Add latitude and longitude to the dataframe
predictions_df.loc[:, 'Longitude'] = lons
predictions_df.loc[:, 'Latitude'] = lats
predictions_df = predictions_df.dropna(subset=['Longitude', 'Latitude', 'MaxTemp'])

# Extract data
lons = predictions_df['Longitude'].values
lats = predictions_df['Latitude'].values
max_temp = predictions_df['MaxTemp'].values
min_temp = predictions_df['MinTemp'].values
rainfall = predictions_df['Rainfall'].values

# Get the date from the predictions_df
forecast_date = predictions_df['Date'].iloc[0]  # Assuming 'Date' column exists and is consistent

# Function to create interpolation map
def create_interpolation_map(ax, lons, lats, values, title, cmap, cbar_label):
    # Define a grid for interpolation
    lon_min, lon_max = lons.min() - 1, lons.max() + 1
    lat_min, lat_max = lats.min() - 1, lats.max() + 1
    lon_grid, lat_grid = np.meshgrid(
        np.linspace(lon_min, lon_max, 200),
        np.linspace(lat_min, lat_max, 200)
    )

    # Perform Ordinary Kriging
    krige = OrdinaryKriging(lons, lats, values, variogram_model='linear')
    grid, _ = krige.execute('grid', lon_grid[0], lat_grid[:, 0])

    # Add features to the map
    ax.add_feature(cfeature.COASTLINE, edgecolor='black')
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.STATES, linestyle=':', edgecolor='gray')

    # Plot interpolated values
    contour = ax.contourf(
        lon_grid, lat_grid, grid,
        levels=20, cmap=cmap, transform=ccrs.PlateCarree()
    )

    # Add data points
    ax.scatter(lons, lats, edgecolors='k', s=50, label='Locations', c='white', transform=ccrs.PlateCarree())

    # Add colorbar
    cbar = plt.colorbar(contour, ax=ax, orientation='vertical')
    cbar.set_label(cbar_label)  # Colorbar only, no black label on map
    ax.set_title(title)

# Create side-by-side maps
fig, axes = plt.subplots(1, 3, figsize=(22, 8), subplot_kw={'projection': ccrs.PlateCarree()})

create_interpolation_map(axes[0], lons, lats, max_temp, "Max Temperature (°C)", 'coolwarm', 'Temperature (°C)')
create_interpolation_map(axes[1], lons, lats, min_temp, "Min Temperature (°C)", 'viridis', 'Temperature (°C)')
create_interpolation_map(axes[2], lons, lats, rainfall, "Rainfall (mm)", 'Blues', 'Rainfall (mm)')

# Add a descriptive label for the entire figure
fig.text(0.5, 0.02, f"Weather Forecast for Australia on {forecast_date}", ha='center', fontsize=14)

# Adjust layout: Add space between the maps
plt.subplots_adjust(wspace=5)  # Increase wspace to add spacing between maps

# Show plot
plt.tight_layout(rect=[0, 0.04, 1, 1])  # Leave space for the bottom label
plt.show()


## Climate diagrams (10 year period)

### X.1 Sydney weather station

In [254]:
from sklearn.linear_model import LinearRegression

## Sydney weather station

# combining year, month, and day into a single Date column
#df_map['Date'] = pd.to_datetime(df_map[['Year', 'Month', 'Day']])

# filtering for Sydney weather station and the required time period
df_sydney = df_map[(df_map['Location'] == 'Sydney') & (df_map['Date'].dt.year >= 2007) & (df_map['Date'].dt.year <= 2017)]

# Sort data by date
df_sydney = df_sydney.sort_values(by='Date')

# Calculate average temperature
df_sydney['AvgTemp'] = (df_sydney['MaxTemp'] + df_sydney['MinTemp']) / 2 
df_sydney['YearMonth'] = df_sydney['Date'].dt.to_period('M')

# Aggregate monthly data
monthly_data = df_sydney.groupby('YearMonth').agg(
    MonthlyMaxTemp=('MaxTemp', 'mean'),
    MonthlyMinTemp=('MinTemp', 'mean'),
    MonthlyAvgTemp=('AvgTemp', 'mean'),
    TotalRainfall=('Rainfall', 'sum')
).reset_index()

# Convert YearMonth to a datetime for plotting
monthly_data['YearMonth'] = monthly_data['YearMonth'].dt.to_timestamp()

# Smooth data using a rolling window
monthly_data['SmoothedMaxTemp'] = monthly_data['MonthlyMaxTemp'].rolling(window=3, center=True).mean()
monthly_data['SmoothedMinTemp'] = monthly_data['MonthlyMinTemp'].rolling(window=3, center=True).mean()
monthly_data['SmoothedAvgTemp'] = monthly_data['MonthlyAvgTemp'].rolling(window=3, center=True).mean()

# Prepare data for linear regression
monthly_data['NumericTime'] = np.arange(len(monthly_data))  # Numeric time for regression (e.g., 0, 1, 2,...)
X = monthly_data[['NumericTime']]
y = monthly_data['MonthlyAvgTemp']

# Fit the linear regression model
reg = LinearRegression()
reg.fit(X, y)

# Predict the trend line
monthly_data['TrendLine'] = reg.predict(X)

# Set up the figure and axes
fig, ax1 = plt.subplots(figsize=(8, 8))

# Plot Smoothed MaxTemp, MinTemp, and AvgTemp as line plots, also overlay regression trend line
ax1.plot(monthly_data['YearMonth'], monthly_data['SmoothedMaxTemp'], label='Smoothed max. temperature (°C)', color='red', linewidth=1)
ax1.plot(monthly_data['YearMonth'], monthly_data['SmoothedMinTemp'], label='Smoothed min. temperature (°C)', color='blue', linewidth=1)
ax1.plot(monthly_data['YearMonth'], monthly_data['SmoothedAvgTemp'], label='Smoothed average temperature (°C)', color='green', linewidth=1)
ax1.plot(monthly_data['YearMonth'], monthly_data['TrendLine'], label='Average temperature trend line', color='purple', linestyle='--', linewidth=1.5)

# Customize primary Y-axis (temperature)
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Temperature (°C)', fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(alpha=0.5)

# Create a secondary Y-axis for Rainfall
ax2 = ax1.twinx()
ax2.bar(
    monthly_data['YearMonth'], 
    monthly_data['TotalRainfall'], 
    color='gray', alpha=0.6, label='Monthly rainfall (mm)', width=20
)

# Customize secondary Y-axis (rainfall)
ax2.set_ylabel('Rainfall (mm)', fontsize=12)
ax2.legend(loc='upper right')

# Title and layout adjustments
plt.title('Weather trends in Sydney (2007–2017)', fontsize=16)
plt.tight_layout()

# Show the plot
plt.show()

# Print regression results
print(f"Slope of the trend line: {reg.coef_[0]:.4f}")
print(f"Intercept of the trend line: {reg.intercept_:.4f}")

Slope of the trend line: 0.0127
Intercept of the trend line: 18.2546


In [256]:
# Driest year
driest_year = (
    df_sydney.groupby(df_sydney['Date'].dt.year)['Rainfall'].sum()
    .idxmin()
)
driest_year_rainfall = df_sydney.groupby(df_sydney['Date'].dt.year)['Rainfall'].sum().min()

# Wettest month
wettest_month = (
    df_sydney.groupby(df_sydney['Date'].dt.to_period('M'))['Rainfall'].sum()
    .idxmax()
)
wettest_month_rainfall = df_sydney.groupby(df_sydney['Date'].dt.to_period('M'))['Rainfall'].sum().max()

# Hottest year
hottest_year = (
    df_sydney.groupby(df_sydney['Date'].dt.year)['AvgTemp'].mean()
    .idxmax()
)
hottest_year_temp = df_sydney.groupby(df_sydney['Date'].dt.year)['AvgTemp'].mean().max()

# Coldest year
coldest_year = (
    df_sydney.groupby(df_sydney['Date'].dt.year)['AvgTemp'].mean()
    .idxmin()
)
coldest_year_temp = df_sydney.groupby(df_sydney['Date'].dt.year)['AvgTemp'].mean().min()

# Monthly averages across all years
monthly_avg = (
    df_sydney.groupby(df_sydney['Date'].dt.month)[['AvgTemp', 'Rainfall']].mean()
)

# Display statistics
print(f"Driest year: {driest_year} with {driest_year_rainfall:.2f} mm of rainfall")
print(f"Wettest month: {wettest_month} with {wettest_month_rainfall:.2f} mm of rainfall")
print(f"Hottest year: {hottest_year} with an average temperature of {hottest_year_temp:.2f} °C")
print(f"Coldest year: {coldest_year} with an average temperature of {coldest_year_temp:.2f} °C")
print("Monthly averages:")
print('-------------------------')
print(monthly_avg)


Driest year: 2017 with 865.80 mm of rainfall
Wettest month: 2015-04 with 366.80 mm of rainfall
Hottest year: 2017 with an average temperature of 20.76 °C
Coldest year: 2008 with an average temperature of 17.71 °C
Monthly averages:
-------------------------
        AvgTemp  Rainfall
Date                     
1     23.916846  3.127599
2     23.533529  4.323922
3     22.369355  4.483871
4     19.440174  5.194815
5     16.632097  2.434194
6     14.436441  5.822240
7     13.489068  2.853047
8     14.535842  2.156272
9     17.305730  1.819855
10    19.123656  2.164734
11    21.144248  2.905347
12    22.192211  2.391935


## X.2 Darwin weather station

In [259]:
from sklearn.linear_model import LinearRegression

## Darwuin weather station

# combining year, month, and day into a single Date column
#df_map['Date'] = pd.to_datetime(df_map[['Year', 'Month', 'Day']])

# filtering for Darwin weather station and the required time period
df_darwin = df_map[(df_map['Location'] == 'Darwin') & (df_map['Date'].dt.year >= 2007) & (df_map['Date'].dt.year <= 2017)]

# Sort data by date
df_darwin = df_darwin.sort_values(by='Date')

# Calculate average temperature
df_darwin['AvgTemp'] = (df_darwin['MaxTemp'] + df_darwin['MinTemp']) / 2 
df_darwin['YearMonth'] = df_darwin['Date'].dt.to_period('M')

# Aggregate monthly data
monthly_data = df_darwin.groupby('YearMonth').agg(
    MonthlyMaxTemp=('MaxTemp', 'mean'),
    MonthlyMinTemp=('MinTemp', 'mean'),
    MonthlyAvgTemp=('AvgTemp', 'mean'),
    TotalRainfall=('Rainfall', 'sum')
).reset_index()

# Convert YearMonth to a datetime for plotting
monthly_data['YearMonth'] = monthly_data['YearMonth'].dt.to_timestamp()

# Smooth data using a rolling window
monthly_data['SmoothedMaxTemp'] = monthly_data['MonthlyMaxTemp'].rolling(window=3, center=True).mean()
monthly_data['SmoothedMinTemp'] = monthly_data['MonthlyMinTemp'].rolling(window=3, center=True).mean()
monthly_data['SmoothedAvgTemp'] = monthly_data['MonthlyAvgTemp'].rolling(window=3, center=True).mean()

# Prepare data for linear regression
monthly_data['NumericTime'] = np.arange(len(monthly_data))  # Numeric time for regression (e.g., 0, 1, 2,...)
X = monthly_data[['NumericTime']]
y = monthly_data['MonthlyAvgTemp']

# Fit the linear regression model
reg = LinearRegression()
reg.fit(X, y)

# Predict the trend line
monthly_data['TrendLine'] = reg.predict(X)

# Set up the figure and axes
fig, ax1 = plt.subplots(figsize=(8, 8))

# Plot Smoothed MaxTemp, MinTemp, and AvgTemp as line plots, also overlay regression trend line
ax1.plot(monthly_data['YearMonth'], monthly_data['SmoothedMaxTemp'], label='Smoothed max. temperature (°C)', color='red', linewidth=1)
ax1.plot(monthly_data['YearMonth'], monthly_data['SmoothedMinTemp'], label='Smoothed min. temperature (°C)', color='blue', linewidth=1)
ax1.plot(monthly_data['YearMonth'], monthly_data['SmoothedAvgTemp'], label='Smoothed average temperature (°C)', color='green', linewidth=1)
ax1.plot(monthly_data['YearMonth'], monthly_data['TrendLine'], label='Average temperature trend line', color='purple', linestyle='--', linewidth=1.5)

# Customize primary Y-axis (temperature)
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Temperature (°C)', fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(alpha=0.5)

# Create a secondary Y-axis for Rainfall
ax2 = ax1.twinx()
ax2.bar(
    monthly_data['YearMonth'], 
    monthly_data['TotalRainfall'], 
    color='gray', alpha=0.6, label='Monthly rainfall (mm)', width=20
)

# Customize secondary Y-axis (rainfall)
ax2.set_ylabel('Rainfall (mm)', fontsize=12)
ax2.legend(loc='upper right')

# Title and layout adjustments
plt.title('Weather trends in Darwin (2007–2017)', fontsize=16)
plt.tight_layout()

# Show the plot
plt.show()

# Print regression results
print(f"Slope of the trend line: {reg.coef_[0]:.4f}")
print(f"Intercept of the trend line: {reg.intercept_:.4f}")

Slope of the trend line: 0.0042
Intercept of the trend line: 27.6526


In [263]:
# Driest year
driest_year = (
    df_darwin.groupby(df_darwin['Date'].dt.year)['Rainfall'].sum()
    .idxmin()
)
driest_year_rainfall = df_darwin.groupby(df_sydney['Date'].dt.year)['Rainfall'].sum().min()

# Wettest month
wettest_month = (
    df_darwin.groupby(df_darwin['Date'].dt.to_period('M'))['Rainfall'].sum()
    .idxmax()
)
wettest_month_rainfall = df_darwin.groupby(df_darwin['Date'].dt.to_period('M'))['Rainfall'].sum().max()

# Hottest year
hottest_year = (
    df_darwin.groupby(df_darwin['Date'].dt.year)['AvgTemp'].mean()
    .idxmax()
)
hottest_year_temp = df_darwin.groupby(df_sydney['Date'].dt.year)['AvgTemp'].mean().max()

# Coldest year
coldest_year = (
    df_darwin.groupby(df_darwin['Date'].dt.year)['AvgTemp'].mean()
    .idxmin()
)
coldest_year_temp = df_darwin.groupby(df_darwin['Date'].dt.year)['AvgTemp'].mean().min()

# Monthly averages across all years
monthly_avg = (
    df_sydney.groupby(df_sydney['Date'].dt.month)[['AvgTemp', 'Rainfall']].mean()
)

# Display statistics
print(f"Driest year: {driest_year} with {driest_year_rainfall:.2f} mm of rainfall")
print(f"Wettest month: {wettest_month} with {wettest_month_rainfall:.2f} mm of rainfall")
print(f"Hottest year: {hottest_year} with an average temperature of {hottest_year_temp:.2f} °C")
print(f"Coldest year: {coldest_year} with an average temperature of {coldest_year_temp:.2f} °C")
print("Monthly averages:")
print('-------------------------')
print(monthly_avg)


Driest year: 2008 with nan mm of rainfall
Wettest month: 2011-02 with 1110.20 mm of rainfall
Hottest year: 2016 with an average temperature of nan °C
Coldest year: 2011 with an average temperature of 26.91 °C
Monthly averages:
-------------------------
        AvgTemp  Rainfall
Date                     
1     23.916846  3.127599
2     23.533529  4.323922
3     22.369355  4.483871
4     19.440174  5.194815
5     16.632097  2.434194
6     14.436441  5.822240
7     13.489068  2.853047
8     14.535842  2.156272
9     17.305730  1.819855
10    19.123656  2.164734
11    21.144248  2.905347
12    22.192211  2.391935


In [274]:
## evaluation

#Comparison of the sliding window weather forecast with the actual weather observations on the date

df_map['Date'] = pd.to_datetime(df_map['Date'])
predictions_df['Date'] = pd.to_datetime(predictions_df['Date'])

# Merge the dataframes on 'Date' and 'Location'
comparison_df = pd.merge(
    predictions_df, 
    df_map, 
    on=['Date', 'Location'], 
    suffixes=('_predicted', '_actual')
)

# Calculate differences
comparison_df['MinTemp_Diff'] = comparison_df['MinTemp_predicted'] - comparison_df['MinTemp_actual']
comparison_df['MaxTemp_Diff'] = comparison_df['MaxTemp_predicted'] - comparison_df['MaxTemp_actual']
comparison_df['Rainfall_Diff'] = comparison_df['Rainfall_predicted'] - comparison_df['Rainfall_actual']

# Group by location and summarize the differences
summary = comparison_df.groupby('Location').agg({
    'MinTemp_Diff': ['mean'],
    'MaxTemp_Diff': ['mean'],
    'Rainfall_Diff': ['mean']
})

# Reset index for easy viewing
summary = summary.reset_index()

prediction_date= predictions_df['Date'].iloc[1]
# Display the summary
print(f"Comparison of Predictions vs Actual Weather by Location on {prediction_date}" )
print(summary)

Comparison of Predictions vs Actual Weather by Location on 2013-01-10 00:00:00
            Location MinTemp_Diff MaxTemp_Diff Rainfall_Diff
                             mean         mean          mean
0           Adelaide     1.239752    -4.473993      0.675066
1             Albany    -0.871493     1.430735      1.925462
2             Albury     0.470822    -2.925068      3.294615
3       AliceSprings     2.264495    -0.078804      0.585651
4      BadgerysCreek    -0.989195     1.131539      4.815527
5           Ballarat    -0.116137    -6.347053      1.293806
6            Bendigo     0.212967    -3.248474      0.934470
7           Brisbane    -1.204417     0.114481      1.158288
8             Cairns     1.031501    -0.348102      7.875951
9           Canberra     3.527491    -1.872712      4.337687
10             Cobar     0.711503    -3.195215      1.928623
11      CoffsHarbour     1.548657     4.030924     -7.798027
12          Dartmoor    -0.699942    -3.104349      0.583980
13    